PostProcessing Workbook

This notebook is for testing calibration and stacking functions in Obsy

First we define our object

In [154]:
import os
from astropy.io import fits
import logging
import sqlite3
import shutil
import uuid
from pathlib import Path
from datetime import datetime
import sys
from math import cos,sin

class PostProcess(object):
    def __init__(self):
        print("PostProcess object created")
        self.DEBUG=True
        self.sourceFolder="/home/gtulloch/obsy/sample_data/Processing/input/"
        self.repoFolder="/home/gtulloch/obsy/sample_data/Processing/repo/"
        self.dbName=dbName = self.repoFolder+"obsy.db"
        # Set up Database
        self.con = sqlite3.connect(dbName)
        self.cur = self.con.cursor()
        self.createTables()

    def createTables(self):
        if self.DEBUG:
            self.cur.execute("DROP TABLE if exists fitsFile")
            self.cur.execute("DROP TABLE if exists fitsHeader")
            self.cur.execute("DROP TABLE if exists fitsSequence")
        self.cur.execute("CREATE TABLE if not exists fitsFile(fitsFileId ,fitsFileName,fitsFileDate,fitsFileCalibrated,fitsFileType,"+ \
                    "fitsFileObject,fitsFileExpTime,fitsFileXBinning, fitsFileYBinning,"+ \
                    "fitsFileCCDTemp, fitsFileTelescop, fitsFileInstrument,fitsFileGain,"+ \
                    "fitsFileOffset,fitsFileStacked,fitsFileSequence)")
        self.cur.execute("CREATE TABLE if not exists fitsHeader(fitsHeaderId,fitsHeaderKey,fitsHeaderValue,fitsFileId)")
        self.cur.execute("CREATE TABLE if not exists fitsSequence(fitsFileSequenceId,fitsFileMasterDark,fitsFileMasterFlat,fitsFileMasterBias)")
        return
    
    def submitFileToDB(self,fileName,hdr):
        if "DATE-OBS" in hdr:
            # Create a uuid for the file
            uuidStr=uuid.uuid4()
            # Save the header data first, extracting appropriate info for the fitsFile table
            objectName,exptime,filterName,xbinning,ybinning,ccdTemp,frameType,telescope,instrument,dateObs,gain,offset="","","","","","","","","","","",""
            for card in hdr:
                if type(hdr[card]) not in [bool,int,float]:
                    keywordValue=str(hdr[card]).replace('\'',' ')
                else:
                    keywordValue = hdr[card]
                    
                if (card=="OBJECT"): 
                    print("Object name is "+keywordValue)
                    objectName=keywordValue
                if (card=="EXPTIME"):
                    exptime=keywordValue    
                if (card=="FILTER"):
                    filterName=keywordValue
                if (card=="XBINNING"):
                    xbinning=keywordValue
                if (card=="YBINNING"):
                    ybinning=keywordValue
                if (card=="CCD-TEMP"):
                    ccdTemp=keywordValue 
                if (card=="FRAME"):
                    frameType=keywordValue
                if (card=="TELESCOP"):
                    telescope=keywordValue
                if (card=="INSTRUME"):
                    instrument=keywordValue
                if (card=="DATE-OBS"):
                    dateObs=keywordValue
                if (card=="GAIN"):
                    gain=keywordValue
                if (card=="OFFSET"):
                    offset=keywordValue

                sqlStmt="INSERT INTO fitsHeader (fitsHeaderId,fitsFileId,fitsHeaderKey,fitsHeaderValue) VALUES ('{0}','{1}','{2}','{3}')".format(uuid.uuid4(),uuidStr,card,keywordValue)

                try:
                    self.cur.execute(sqlStmt)
                    self.con.commit()
                except sqlite3.Error as er:
                    exc_type, exc_value, exc_tb = sys.exc_info()

            # We have what we need to save the file to the database
            sqlStmt="INSERT INTO fitsFile (fitsFileId ,fitsFileName,fitsFileDate,fitsFileType,fitsFileObject,fitsFileExpTime,fitsFileXBinning, fitsFileYBinning,"+ \
                    "fitsFileCCDTemp, fitsFileTelescop, fitsFileInstrument,fitsFileGain,fitsFileOffset)"+ \
                    "VALUES ('{0}','{1}','{2}','{3}','{4}','{5}','{6}','{7}','{8}','{9}','{10}','{11}','{12}')" \
                    .format(uuidStr,hdr["DATE-OBS"],fileName,frameType,objectName,exptime,xbinning,ybinning,ccdTemp,telescope,instrument,gain,offset)
            print(sqlStmt)
            self.cur.execute(sqlStmt)
            self.con.commit()
                


        else:
            print("Error: File not added to repo due to missing date is "+fileName)
            return 0
        
        return 1
    
    def registerFitsImages(self):
        # Scan the pictures folder
        print("Processing images in "+self.sourceFolder)
        for root, dirs, files in os.walk(os.path.abspath(self.sourceFolder)):
            for file in files:
                print("Processing file "+os.path.join(root, file))
                file_name, file_extension = os.path.splitext(os.path.join(root, file))

                # Ignore everything not a *fit* file
                if "fit" not in file_extension:
                    print("Ignoring file "+os.path.join(root, file)+" with extension -"+file_extension+"-")
                    continue

                try:
                    hdul = fits.open(os.path.join(root, file), mode='update')
                except ValueError as e:
                    print("Invalid FITS file. File not processed is "+str(os.path.join(root, file)))
                    continue   
        
                hdr = hdul[0].header
                if "FRAME" in hdr:
                    # Create an os-friendly date
                    try:
                        if "DATE-OBS" not in hdr:
                            print("No DATE-OBS card in header. File not processed is "+str(os.path.join(root, file)))
                            continue
                        datestr=hdr["DATE-OBS"].replace("T", " ")
                        datestr=datestr[0:datestr.find('.')]
                        dateobj=datetime.strptime(datestr, '%Y-%m-%d %H:%M:%S')
                        fitsDate=dateobj.strftime("%Y%m%d%H%M%S")
                    except ValueError as e:
                        print("Invalid date format in header. File not processed is "+str(os.path.join(root, file)))
                        continue

                    # Create a new standard name for the file based on what it is
                    if (hdr["FRAME"]=="Light"):
                        # Adjust the WCS for the image
                        if "CD1_1" not in hdr:
                            if "CDELT1" in hdr:
                                fitsCDELT1=float(hdr["CDELT1"])
                                fitsCDELT2=float(hdr["CDELT2"])
                                fitsCROTA2=float(hdr["CROTA2"])
                                fitsCD1_1 =  fitsCDELT1 * cos(fitsCROTA2)
                                fitsCD1_2 = -fitsCDELT2 * sin(fitsCROTA2)
                                fitsCD2_1 =  fitsCDELT1 * sin (fitsCROTA2)
                                fitsCD2_2 = fitsCDELT2 * cos(fitsCROTA2)
                                hdr.append(('CD1_1', str(fitsCD1_1), 'Adjusted via MCP'), end=True)
                                hdr.append(('CD1_2', str(fitsCD1_2), 'Adjusted via MCP'), end=True)
                                hdr.append(('CD2_1', str(fitsCD2_1), 'Adjusted via MCP'), end=True)
                                hdr.append(('CD2_2', str(fitsCD2_2), 'Adjusted via MCP'), end=True)
                                hdul.flush()  # changes are written back to original.fits
                            else:
                                print("No WCS information in header, file not updated is "+str(os.path.join(root, file)))

                        # Assign a new name
                        if ("OBJECT" in hdr):
                            if ("FILTER" in hdr):
                                newName="{0}-{1}-{2}-{3}-{4}-{5}s-{6}x{7}-t{8}.fits".format(hdr["OBJECT"].replace(" ", "_"),hdr["TELESCOP"].replace(" ", "_").replace("\\", "_"),
                                            hdr["INSTRUME"].replace(" ", "_"),hdr["FILTER"],fitsDate,hdr["EXPTIME"],hdr["XBINNING"],hdr["YBINNING"],hdr["CCD-TEMP"])
                            else:
                                newName=newName="{0}-{1}-{2}-{3}-{4}-{5}s-{6}x{7}-t{8}.fits".format(hdr["OBJECT"].replace(" ", "_"),hdr["TELESCOP"].replace(" ", "_").replace("\\", "_"),
                                            hdr["INSTRUME"].replace(" ", "_"),"OSC",fitsDate,hdr["EXPTIME"],hdr["XBINNING"],hdr["YBINNING"],hdr["CCD-TEMP"])
                        else:
                            print("Invalid object name in header. File not processed is "+str(os.path.join(root, file)))
                            continue
                    elif hdr["FRAME"]=="Flat":
                        if ("FILTER" in hdr):
                            newName="{0}-{1}-{2}-{3}-{4}-{5}s-{6}x{7}-t{8}.fits".format(hdr["FRAME"],hdr["TELESCOP"].replace(" ", "_").replace("\\", "_"),
                                            hdr["INSTRUME"].replace(" ", "_"),hdr["FILTER"],fitsDate,hdr["EXPTIME"],hdr["XBINNING"],hdr["YBINNING"],hdr["CCD-TEMP"])
                        else:
                            newName=newName="{0}-{1}-{2}-{3}-{4}-{5}s-{6}x{7}-t{8}.fits".format(hdr["FRAME"],hdr["TELESCOP"].replace(" ", "_").replace("\\", "_"),
                                            hdr["INSTRUME"].replace(" ", "_"),"OSC",fitsDate,hdr["EXPTIME"],hdr["XBINNING"],hdr["YBINNING"],hdr["CCD-TEMP"])
                    elif hdr["FRAME"]=="Dark" or hdr["FRAME"]=="Bias":
                        newName="{0}-{1}-{1}-{2}-{3}s-{4}x{5}-t{6}.fits".format(hdr["FRAME"],hdr["TELESCOP"].replace(" ", "_").replace("\\", "_"),
                                            hdr["INSTRUME"].replace(" ", "_"),fitsDate,hdr["EXPTIME"],hdr["XBINNING"],hdr["YBINNING"],hdr["CCD-TEMP"])
                    else:
                        print("File not processed as FRAME not recognized: "+str(os.path.join(root, file)))
                    hdul.close()

                    # Create the folder structure (if needed)
                    fitsDate=dateobj.strftime("%Y%m%d")
                    if (hdr["FRAME"]=="Light"):
                        newPath=self.repoFolder+"Light/{0}/{1}/{2}/{3}/".format(hdr["OBJECT"].replace(" ", ""),hdr["TELESCOP"].replace(" ", "_").replace("\\", "_"),
                                            hdr["INSTRUME"].replace(" ", "_"),fitsDate)
                    elif hdr["FRAME"]=="Dark":
                        newPath=self.repoFolder+"Calibrate/{0}/{1}/{2}/{3}/{4}/".format(hdr["FRAME"],hdr["TELESCOP"].replace(" ", "_").replace("\\", "_"),
                                            hdr["INSTRUME"].replace(" ", "_"),hdr["EXPTIME"],fitsDate)
                    elif hdr["FRAME"]=="Flat":
                        if ("FILTER" in hdr):
                            newPath=self.repoFolder+"Calibrate/{0}/{1}/{2}/{3}/{4}/".format(hdr["FRAME"],hdr["TELESCOP"].replace(" ", "_").replace("\\", "_"),
                                            hdr["INSTRUME"].replace(" ", "_"),hdr["FILTER"],fitsDate)
                        else:
                            newPath=self.repoFolder+"Calibrate/{0}/{1}/{2}/{3}/{4}/".format(hdr["FRAME"],hdr["TELESCOP"].replace(" ", "_").replace("\\", "_"),
                                            hdr["INSTRUME"].replace(" ", "_"),"OSC",fitsDate)
                    elif hdr["FRAME"]=="Bias":
                        newPath=self.repoFolder+"Calibrate/{0}/{1}/{2}/{3}/".format(hdr["FRAME"],hdr["TELESCOP"].replace(" ", "_").replace("\\", "_"),
                                            hdr["INSTRUME"].replace(" ", "_"),fitsDate)

                    if not os.path.isdir(newPath):
                        os.makedirs (newPath)

                    # If we can add the file to the database move it to the repo
                    if (self.submitFileToDB(newPath+newName.replace(" ", "_"),hdr)):
                        moveInfo="Moving {0} to {1}\n".format(os.path.join(root, file),newPath+newName)
                        print(moveInfo)
                        #shutil.move(os.path.join(root, file),newPath+newName)
                    else:
                        print("Warning: File not added to repo is "+str(os.path.join(root, file)))
                else:
                    print("File not added to repo - no FRAME card - "+str(os.path.join(root, file)))
        return
    

Run the register program to populate the database. 

In [ ]:
postProcessObj=PostProcess()
postProcessObj.registerFitsImages()

Lets see what was imported

In [ ]:
postProcessObj.cur.execute("SELECT * FROM fitsFile LIMIT 10")
files=postProcessObj.cur.fetchall()
print(files)
postProcessObj.cur.execute("SELECT * FROM fitsHeader LIMIT 10")
files=postProcessObj.cur.fetchall()
print(files)

We have a repository, lets create sequences for which we have not already done so. A sequence will include the lights that are taken of the same object in the same session, as well as the darks, flats, and biases associated with them.

In [166]:
postProcessObj.cur.execute("SELECT * FROM fitsFile WHERE fitsFileType='Light' AND fitsFileSequence is NULL ORDER BY fitsFileDate")
files=postProcessObj.cur.fetchall()
currObject=None
uuidStr=uuid.uuid4()
for fitsFile in files:
    if fitsFile[5] != currObject:
        currObject=fitsFile[5]
        print("New object "+currObject)
        uuidStr=uuid.uuid4() # New sequence
    sqlStmt="UPDATE fitsFile SET fitsFileSequence='{0}' WHERE fitsFileId='{1}'".format(uuidStr,fitsFile[0])
    postProcessObj.cur.execute(sqlStmt)
    postProcessObj.con.commit()        
    print("Set sequence for light "+fitsFile[0]+" to "+str(uuidStr))
    postProcessObj.con.commit()     



Same for darks, flats, and biases

In [179]:
from datetime import datetime, timedelta

def sameDay(Date1,Date2): # If Date1 is within 12 hours of Date2
    current_date = datetime.strptime(Date1, '%Y-%m-%d')
    target_date = datetime.strptime(Date2, '%Y-%m-%d')
    time_difference = abs(current_date - target_date)
    return time_difference <= timedelta(hours=12)

# Set sequences for calibration files
# Darks
postProcessObj.cur.execute("SELECT * FROM fitsFile WHERE fitsFileType='Dark'  ORDER BY fitsFileDate")
files=postProcessObj.cur.fetchall()
print(files)
uuidStr=uuid.uuid4()
currDate="0001-01-01"
for fitsFile in files:
    if not sameDay(fitsFile[1][0:fitsFile[1].find('T')],currDate):
        currDate=fitsFile[1][0:fitsFile[1].find('T')]
        uuidStr=uuid.uuid4() # New sequence
        print("New date "+currDate) 
    sqlStmt="UPDATE fitsFile SET fitsFileSequence='{0}' WHERE fitsFileId='{1}'".format(uuidStr,fitsFile[0])
    postProcessObj.cur.execute(sqlStmt)
    postProcessObj.con.commit()        
    print("Set sequence for dark "+fitsFile[0]+" to "+str(uuidStr)) 

# Flats
postProcessObj.cur.execute("SELECT * FROM fitsFile WHERE fitsFileType='Flat'  ORDER BY fitsFileDate")
files=postProcessObj.cur.fetchall()
print(files)
uuidStr=uuid.uuid4()
currDate="0001-01-01"
for fitsFile in files:
    if not sameDay(fitsFile[1][0:fitsFile[1].find('T')],currDate):
        currDate=fitsFile[1][0:fitsFile[1].find('T')]
        uuidStr=uuid.uuid4() # New sequence
        print("New date "+currDate) 
    sqlStmt="UPDATE fitsFile SET fitsFileSequence='{0}' WHERE fitsFileId='{1}'".format(uuidStr,fitsFile[0])
    postProcessObj.cur.execute(sqlStmt)
    postProcessObj.con.commit()        
    print("Set sequence for flat "+fitsFile[0]+" to "+str(uuidStr)) 

# Biases
postProcessObj.cur.execute("SELECT * FROM fitsFile WHERE fitsFileType='Bias'  ORDER BY fitsFileDate")
files=postProcessObj.cur.fetchall()
print(files)
uuidStr=uuid.uuid4()
currDate="0001-01-01"
for fitsFile in files:
    if not sameDay(fitsFile[1][0:fitsFile[1].find('T')],currDate):
        currDate=fitsFile[1][0:fitsFile[1].find('T')]
        uuidStr=uuid.uuid4() # New sequence
        print("New date "+currDate) 
    sqlStmt="UPDATE fitsFile SET fitsFileSequence='{0}' WHERE fitsFileId='{1}'".format(uuidStr,fitsFile[0])
    postProcessObj.cur.execute(sqlStmt)
    postProcessObj.con.commit()        
    print("Set sequence for bias "+fitsFile[0]+" to "+str(uuidStr)) 

[('3a8d1edf-3b92-46ce-8560-7b8eed634d39', '2024-05-30T03:30:59.595', '/home/gtulloch/obsy/sample_data/Processing/repo/Calibrate/Dark/Explore_Scientific_FL-AR102600TN_600@F_5.9/ZWO_CCD_ASI294MC/60.0/20240530/Dark-Explore_Scientific_FL-AR102600TN_600@F_5.9-Explore_Scientific_FL-AR102600TN_600@F_5.9-ZWO_CCD_ASI294MC-20240530033059s-60.0x1-t1.fits', None, 'Dark', '', '60.0', '1', '1', '24.6', 'Explore Scientific FL-AR102600TN 600@F\\5.9', 'ZWO CCD ASI294MC', '115.0', '32.0', None, '9e860b62-ce46-4a16-b2b3-f70164a728d4'), ('91bc9b58-656c-4341-bc64-d4eb72a33025', '2024-05-30T03:32:01.090', '/home/gtulloch/obsy/sample_data/Processing/repo/Calibrate/Dark/Explore_Scientific_FL-AR102600TN_600@F_5.9/ZWO_CCD_ASI294MC/60.0/20240530/Dark-Explore_Scientific_FL-AR102600TN_600@F_5.9-Explore_Scientific_FL-AR102600TN_600@F_5.9-ZWO_CCD_ASI294MC-20240530033201s-60.0x1-t1.fits', None, 'Dark', '', '60.0', '1', '1', '24.6', 'Explore Scientific FL-AR102600TN 600@F\\5.9', 'ZWO CCD ASI294MC', '115.0', '32.0', No

Now we have sequences generated we need to determine if masters exist for each sequence. If not, generate them with pySiril

In [194]:
def assignMasterDark(fitsFileArray):
    pass

def assignMasterFlat(fitsFileArray):
    pass

def assignMasterBias(fitsFileArray):
    pass
    
postProcessObj.cur.execute("SELECT * FROM fitsFile WHERE fitsFileType='Light' AND fitsFileCalibrated is NULL ORDER BY fitsFileSequence")
files=postProcessObj.cur.fetchall()

lastSequence=""
for fitsFile in files:
    print("Calibrating object "+fitsFile[5])

    # Find appropriate sequence record to match masters
    postProcessObj.cur.execute("SELECT * FROM fitsSequence WHERE fitsFileSequenceId='{0}'".format(fitsFile[7]))
    sequenceRec=postProcessObj.cur.fetchone()
    if sequenceRec:
        print("Sequence record found, updating sequence record"+sequenceRec[0])
        newRecord=False
    else:
        print("No sequence record found, creating one")
        newRecord=True
        sequenceRec=[fitsFile[0],None,None,None]

    if sequenceRec[1]==None: # Find or Create Master Dark, then apply it
        print("Creating Master Dark")
        assignMasterDark(fitsFile)
    if sequenceRec[2]==None: # Find or Create Master Flat, then apply it
        print("Creating Master Flat")
        assignMasterFlat(fitsFile)
    if sequenceRec[3]==None: # Find or Create Master Bias, then apply it
        print("Creating Master Bias")
        assignMasterBias(fitsFile)

    # Create or update record
    #if newRecord:
    #    postProcessObj.cur.execute("INSERT INTO fitsSequence (fitsFileSequenceId, fitsFileSequenceType, fitsFileSequenceObject, fitsFileSequenceFilter, fitsFileSequenceExposure, fitsFileSequenceCount, fitsFileSequenceStart, fitsFileSequenceEnd, fitsFileSequenceStatus) VALUES ('{0}', '{1}', '{2}', '{3}', '{4}', '{5}', '{6}', '{7}', '{8}')".format(fitsFile[7], fitsFile[2], fitsFile[5], fitsFile[3], fitsFile[4], 1, fitsFile[0], fitsFile[0], 'Calibrating'))
    #else:
    #    postProcessObj.cur.execute("UPDATE fitsSequence SET fitsFileSequenceEnd='{0}', fitsFileSequenceCount=fitsFileSequenceCount+1 WHERE fitsFileSequence='{1}'".format(fitsFile[0], sequenceRec[0]))


    


Calibrating object NGC 7822
No sequence record found, creating one
Creating Master Dark
Creating Master Flat
Creating Master Bias
Calibrating object NGC 7822
No sequence record found, creating one
Creating Master Dark
Creating Master Flat
Creating Master Bias
Calibrating object NGC 7822
No sequence record found, creating one
Creating Master Dark
Creating Master Flat
Creating Master Bias
Calibrating object NGC 7822
No sequence record found, creating one
Creating Master Dark
Creating Master Flat
Creating Master Bias
Calibrating object NGC 7822
No sequence record found, creating one
Creating Master Dark
Creating Master Flat
Creating Master Bias
Calibrating object NGC 7822
No sequence record found, creating one
Creating Master Dark
Creating Master Flat
Creating Master Bias
Calibrating object NGC 7822
No sequence record found, creating one
Creating Master Dark
Creating Master Flat
Creating Master Bias
Calibrating object NGC 7822
No sequence record found, creating one
Creating Master Dark
Cr